In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import jaccard_score
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/oignat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/oignat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/oignat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv("data/all_data.csv")

In [3]:
df['uk'] = (df['Country'] == 'uk').astype(int)
df['india'] = (df['Country'] == 'india').astype(int)
df['real'] = (df['Source'] == 'real').astype(int)
df['generated'] = (df['Source'] == 'generated').astype(int)

In [4]:
df

,Text,Country,Source,Label,uk,india,real,generated
0,Palaeontologist.,uk,real,0,1,0,1,0
1,\n> We have the capability of being better. B...,uk,real,1,1,0,1,0
2,"Only just noticed it myself, ha! Or Ha, accord...",uk,real,0,1,0,1,0
3,Mate all he does is play football and misses p...,uk,real,1,1,0,1,0
4,"Me neither. But then, I'm not sure Cleese woul...",uk,real,0,1,0,1,0
...,...,...,...,...,...,...,...,...
5995,"Often, I find my inspiration in the diverse an...",india,generated,1,0,1,0,1
5996,I often find inspiration in people overcoming ...,india,generated,1,0,1,0,1
5997,What inspires me most is the sheer determinati...,india,generated,1,0,1,0,1
5998,"Just the other day, I watched a fascinating do...",india,generated,1,0,1,0,1


In [5]:
def decontract(sentence):
    sentence = re.sub(r"n\'t", " not", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'s", " is", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'t", " not", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'m", " am", sentence)
    return sentence

In [6]:
def remove_emojis(sentence):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               "]+", flags=re.UNICODE)
    # Remove emojis from the text
    text_without_emojis = emoji_pattern.sub(r'', sentence)
    return text_without_emojis

In [7]:
def removePunctuation(sentence):
    sentence = re.sub(r'[?|!|\'|"|#|>|<]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence)
    sentence = sentence.strip()
    sentence = sentence.replace("\n"," ")
    return sentence

In [8]:
def removeStopWords(sentence):
    stop_words = set(stopwords.words('english'))
    text_without_stopwords = ' '.join(word for word in sentence.split() if word.lower() not in stop_words)
    return sentence  # Join the words back into a sentence

In [9]:
def lemmatize(sentence):
  lemmatizer = WordNetLemmatizer()
  lemmatizedSentence = ""
  for word in sentence.split():
        lemm = lemmatizer.lemmatize(word)
        lemmatizedSentence += lemm
        lemmatizedSentence += " "
  lemmatizedSentence = lemmatizedSentence.strip()
  return lemmatizedSentence

In [10]:
def preprocess(sentence):
  sentence = sentence.lower()
  sentence = decontract(sentence)
  sentence = remove_emojis(sentence)
  sentence = removePunctuation(sentence)
  sentence = removeStopWords(sentence)
  sentence = lemmatize(sentence)
  return sentence

In [11]:
df = df.dropna(subset=['Text'])
df['processed_text'] = df['Text'].apply(preprocess)
df.head()

/tmp/ipykernel_1042013/1147288075.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['Text'].apply(preprocess)


,Text,Country,Source,Label,uk,india,real,generated,processed_text
0,Palaeontologist.,uk,real,0,1,0,1,0,palaeontologist
1,\n> We have the capability of being better. B...,uk,real,1,1,0,1,0,we have the capability of being better but we ...
2,"Only just noticed it myself, ha! Or Ha, accord...",uk,real,0,1,0,1,0,only just noticed it myself ha or ha according...
3,Mate all he does is play football and misses p...,uk,real,1,1,0,1,0,mate all he doe is play football and miss pen ...
4,"Me neither. But then, I'm not sure Cleese woul...",uk,real,0,1,0,1,0,me neither but then i am not sure cleese would...


In [27]:
#TODO: choose few-shot or default test-train split
X_train, X_test, y_train, y_test = train_test_split(df['processed_text'], df[df.columns[3:8]], test_size=0.2, random_state=42, shuffle=True)
# X_train, X_test, y_train, y_test = train_test_split(df['processed_text'], df[df.columns[3:8]], test_size=0.9, random_state=42, shuffle=True)
print(X_train.shape, X_test.shape)

(4799,) (1200,)


In [28]:
vectorizer = TfidfVectorizer(strip_accents='unicode', max_features = 4500)
# vectorizer = TfidfVectorizer(max_features = 4500)

In [29]:
vectorizer.fit(X_train)

TfidfVectorizer(max_features=4500, strip_accents='unicode')

In [30]:
X_train_tfidf = vectorizer.transform(X_train).todense()
X_test_tfidf = vectorizer.transform(X_test).todense()

In [31]:
y_test

,Label,uk,india,real,generated
84,0,1,0,1,0
2409,0,0,1,1,0
4997,1,1,0,0,1
346,1,1,0,1,0
79,0,1,0,1,0
...,...,...,...,...,...
4274,1,1,0,0,1
4901,1,1,0,0,1
2997,1,0,1,1,0
5113,1,0,1,0,1


### Random Forest classifier

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.impute import SimpleImputer

def run_model(X_train, X_test, model):
    imp = SimpleImputer(missing_values=np.nan, strategy='mean') # replace missing values with mean
    imp = imp.fit(X_train)
    X_train = imp.transform(X_train)
    X_test = imp.transform(X_test)

    clf = model.fit(X_train, y_train['Label']) # Label is inspirational?
    predicted_inspir = clf.predict(X_test)

    clf = model.fit(X_train, y_train['generated']) # is generated?
    predicted_gen = clf.predict(X_test)

    clf = model.fit(X_train, y_train['real']) # is real?
    predicted_real = clf.predict(X_test)

    clf = model.fit(X_train, y_train['uk']) # is uk?
    predicted_uk = clf.predict(X_test)

    clf = model.fit(X_train, y_train['india']) # is india?
    predicted_india = clf.predict(X_test)
    
    return predicted_inspir, predicted_gen, predicted_real, predicted_uk, predicted_india

### Style Features

In [35]:
%pip install textdescriptives
# %pip install better_profanity
!python3 -m spacy download en_core_web_lg

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 254 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 62.8 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 127.2 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 1.7 MB/s s eta 0:00:01
     |████████████████████████████████| 409 kB 134.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 119.7 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 2.8 MB/s s eta 0:00:01
     |████████████████████████████████| 50 kB 4.1 MB/s  eta 0:00:01
     |████████████████████████████████| 492 kB 76.0 MB/s eta 0:00:01
     |████████████████████████████████| 157 kB 124.3 MB/s eta 0:00:01
     |████████████████████████████████| 937 kB 101.9 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 2.9 MB/s s eta 0:00:01
     |████████████████████████████████| 182 kB 130.5 MB/s eta 0:00:01
     |███████████████

Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████▊               | 306.7 MB 113.7 MB/s eta 0:00:03

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████▌        | 432.6 MB 120.9 MB/s eta 0:00:02

     |███████████████████████▉        | 438.6 MB 120.9 MB/s eta 0:00:02

...

     |███████████████████████████████ | 571.3 MB 122.7 MB/s eta 0:00:01

     |███████████████████████████████▍| 577.3 MB 122.7 MB/s eta 0:00:01

     |███████████████████████████████▊| 583.4 MB 122.7 MB/s eta 0:00:01

     |████████████████████████████████| 587.7 MB 120.0 MB/s eta 0:00:01

     |████████████████████████████████| 587.7 MB 14 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [56]:
import textdescriptives as td #https://github.com/HLasse/TextDescriptives
import spacy
spacy.load('en_core_web_lg')
td.get_valid_metrics()

{'all',
 'coherence',
 'dependency_distance',
 'descriptive_stats',
 'information_theory',
 'pos_proportions',
 'quality',
 'readability'}

In [36]:
df = td.extract_metrics(text=X_train, lang='en', metrics=["pos_proportions"])
descr_train = df['pos_prop_ADJ']
df = td.extract_metrics(text=X_test, lang='en', metrics=["pos_proportions"])
descr_test = df['pos_prop_ADJ']

df = td.extract_metrics(text=X_train, lang='en', metrics=["readability"])
readability_train = df['flesch_reading_ease']
df = td.extract_metrics(text=X_test, lang='en', metrics=["readability"])
readability_test = df['flesch_reading_ease']

NameError: name 'td' is not defined

In [ ]:
# All scores
X_train_scores_r = np.asarray(readability_train).reshape(-1, 1)
X_test_scores_r = np.asarray(readability_test).reshape(-1, 1) 

X_train_scores_d = np.asarray(descr_train).reshape(-1, 1)
X_test_scores_d = np.asarray(descr_test).reshape(-1, 1) 
print(X_train_scores_r.shape, X_train_vec.shape)

In [34]:
import copy
# X_train_all = np.concatenate((X_train_tfidf, X_train_scores_d, X_train_scores_r), axis=1)
# X_test_all = np.concatenate((X_test_tfidf, X_test_scores_d, X_test_scores_r), axis=1)

# X_train_scores = np.concatenate((X_train_scores_d, X_train_scores_r), axis=1)
# X_test_scores = np.concatenate((X_test_scores_d, X_test_scores_r), axis=1)

# X_train = copy.deepcopy(X_train_all) 
# X_test =  copy.deepcopy(X_test_all) 

X_train = copy.deepcopy(X_train_tfidf) 
X_test =  copy.deepcopy(X_test_tfidf) 

# X_train = copy.deepcopy(X_train_scores) 
# X_test =  copy.deepcopy(X_test_scores)

In [35]:
model = RandomForestClassifier()
# model = SVC()
# model = GaussianNB()
predicted_inspir, predicted_gen, predicted_real, predicted_uk, predicted_india = run_model(X_train, X_test, model)

# results
print("\'Inspiring\': " + str(multi_label_metrics(predicted_inspir, y_test['Label'])))
print("\'Generated\': " + str(multi_label_metrics(predicted_gen, y_test['generated'])))
print("\'Real\': " + str(multi_label_metrics(predicted_real, y_test['real'])))
print("\'UK\': " + str(multi_label_metrics(predicted_uk, y_test['uk'])))
print("\'India\': " + str(multi_label_metrics(predicted_india, y_test['india'])))

'Inspiring': {'P': 92.5280199252802, 'R': 93.45911949685535, 'F1': 90.66666666666666}
'Generated': {'P': 98.73096446700508, 'R': 99.48849104859335, 'F1': 99.41666666666666}
'Real': {'P': 99.50372208436724, 'R': 99.38042131350682, 'F1': 99.25}
'UK': {'P': 90.68627450980392, 'R': 88.9423076923077, 'F1': 89.5}
'India': {'P': 88.26530612244898, 'R': 90.73426573426573, 'F1': 89.83333333333333}


### Evaluation

In [33]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score

def multi_label_metrics(predicted, labels):
    y_true=predicted
    y_pred=labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    
    # return as dictionary
    metrics = {'P': precision * 100,
               'R': recall * 100,
               'F1': f1_micro_average *100,}
    return metrics